In [1]:
import warnings
warnings.filterwarnings('ignore')
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import re, string, unicodedata
import nltk
#nltk.download('popular')
from nltk.stem.snowball import SnowballStemmer
#import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from numpy import mean, std
from sklearn.metrics import classification_report
import plotly.graph_objs as go
import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
import matplotlib.pyplot as plt




Using TensorFlow backend.


In [21]:
data = pd.read_csv('google-maps-single-dataset-for-conference.csv',encoding='ISO-8859-1')
data= data[pd.notnull(data['Rationale_Type'])]
data = data.replace("issue ", "issue")
print(len(data))
data.head()

3051


,comment_Text,comment_author,comment_id,comment_upvotes,parent_id,Rationale_Type,Claim_Type,Have_Rationale
1,People still do racial checkbox ads?,PubicLouseInDaHouse,dfadazi,-1.0,60v5za,claim,neu,no
2,What?,Isvara,dfade5q,1.0,dfadazi,claim,neu,no
3,90s power rangers.,PubicLouseInDaHouse,dfadlgi,1.0,dfade5q,claim,neu,no
4,Are there really uses for that?,piponwa,dfa8rlu,-1.0,60v5za,issue,NaN,yes
5,https://support.google.com/plus/answer/2998354...,sparr,dfa365t,-1.0,60v5za,claim,supporting,yes


In [22]:
def print_plot(index):
    example = data[data.index == index][['comment_Text', 'Rationale_Type']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Rationale_Type:', example[1])


In [23]:
print_plot(60)

> gender equality in any and all professions.

Hahah uh... what?

EDIT: Also I wasn't the one to downvote you above. No clue what that's about.
Rationale_Type: claim


In [24]:
#df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\<\>\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z =#+_]')
#STOPWORDS = set(stopwords.words('english'))

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_url(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = strip_html(text) 
    text = remove_url(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['comment_Text'] = data['comment_Text'].apply(clean_text)
#df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')

In [25]:
print_plot(60)

  gender equality in any and all professionshahah uh whatedit also i wasnt the one to downvote you above no clue what thats about
Rationale_Type: claim


In [26]:
print(data.Rationale_Type.value_counts())
X= data.comment_Text.values.astype('U')
y=data.Rationale_Type.values.astype('U')
rationale_type=data.Rationale_Type
rationale_type_name= data.Rationale_Type.unique()
print(rationale_type_name)

claim      2076
feature     667
issue       308
Name: Rationale_Type, dtype: int64
['claim' 'issue' 'feature']


In [27]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_tfidf= tokenizer.texts_to_matrix(trainX,mode='tfidf')
    X_test_matrix_tfidf= tokenizer.texts_to_matrix(testX, mode='tfidf')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_tfidf= encoder.transform(trainy)
    Y_test_matrix_tfidf= encoder.transform(testy)
    smote = SMOTE()
    X_sm_tfidf, y_sm_tfidf = smote.fit_sample(X_train_matrix_tfidf, Y_train_matrix_tfidf)
    
    #print(vocab_size)
    model = Sequential()
    #input layer
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.81))
    #hidden layer
    #odel.add(Dense(32))
    #odel.add(Activation('relu'))
    #odel.add(Dropout(0.3))
    #output layer
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    #model. summary()
    #print(model.metrics_names)
    #fit model 
    model.fit(X_sm_tfidf,y_sm_tfidf, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
    #evaluate model
    score= model.evaluate(X_test_matrix_tfidf, Y_test_matrix_tfidf, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_tfidf, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_tfidf, axis=1), predicted)
    return model, test_accuracy, report_accuracy



    

In [28]:
#Kfold ANN with TFIDF

kf= KFold(n_splits=3)
#curr_fold= 0
alg_accurcy_ANN_tfidf =[]
alg_testing_accurcy_ANN_tfidf =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_tfidf.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_tfidf.append(report_accuracy)
    

average_accuracy_ANN_training= np.mean(alg_accurcy_ANN_tfidf, axis=0)
print("the average Training Accuracy of ANN-TFIDF is",average_accuracy_ANN_training) 

average_accuracy_ANN_testing= np.mean(alg_testing_accurcy_ANN_tfidf, axis=0)
print("the averag Validation accuracy of ANN-TFIDF is",average_accuracy_ANN_testing)



Train on 3283 samples, validate on 821 samples
Epoch 1/10
3283/3283 [==============================] - 9s 3ms/step - loss: 1.0487 - acc: 0.4496 - val_loss: 1.2456 - val_acc: 0.1705
Epoch 2/10
3283/3283 [==============================] - 2s 611us/step - loss: 0.8916 - acc: 0.5815 - val_loss: 1.0030 - val_acc: 0.5493
Epoch 3/10
3283/3283 [==============================] - 2s 643us/step - loss: 0.7935 - acc: 0.6339 - val_loss: 0.7751 - val_acc: 0.7247
Epoch 4/10
3283/3283 [==============================] - 2s 527us/step - loss: 0.7016 - acc: 0.6680 - val_loss: 0.6439 - val_acc: 0.7747
Epoch 5/10
3283/3283 [==============================] - 2s 519us/step - loss: 0.6779 - acc: 0.6762 - val_loss: 0.5477 - val_acc: 0.8088
Epoch 6/10
3283/3283 [==============================] - 2s 516us/step - loss: 0.6440 - acc: 0.7048 - val_loss: 0.4912 - val_acc: 0.8343
Epoch 7/10
3283/3283 [==============================] - 2s 567us/step - loss: 0.6104 - acc: 0.7091 - val_loss: 0.4561 - val_acc: 0.8307
Epo

In [29]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    
    
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_binary= tokenizer.texts_to_matrix(trainX,mode='binary')
    X_test_matrix_binary= tokenizer.texts_to_matrix(testX, mode='binary')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_binary= encoder.transform(trainy)
    Y_test_matrix_binary= encoder.transform(testy)
    
    
    smote = SMOTE()
    X_sm_binary, y_sm_binary = smote.fit_sample(X_train_matrix_binary, Y_train_matrix_binary)
    
    MAX_SEQUENCE_LENGTH = 1000
    #print(vocab_size)
    model = Sequential()
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))
    #hidden layer
    #model.add(Dense(16))
    #model.add(Activation('relu'))
    #model.add(Dropout(0.5))

    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    #model. summary()
    #print(model.metrics_names)

    model.fit(X_sm_binary,y_sm_binary, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
    score= model.evaluate(X_test_matrix_binary, Y_test_matrix_binary, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_binary, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_binary, axis=1), predicted)
    return model, test_accuracy, report_accuracy  

    

In [30]:
#Kfold ANN with mode=binary

kf= KFold(n_splits=2)
#curr_fold= 0
alg_accurcy_ANN_binary =[]
alg_testing_accurcy_ANN_binary =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_binary.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_binary.append(report_accuracy)
    

average_accuracy_ANN_binary= np.mean(alg_accurcy_ANN_tfidf, axis=0)
print("the average Training Accuracy of ANN-Binary is",average_accuracy_ANN_binary) 

average_accuracy_ANN_binary= np.mean(alg_testing_accurcy_ANN_tfidf, axis=0)
print("the averag Validation accuracy of ANN-Binary is",average_accuracy_ANN_binary)





Train on 2517 samples, validate on 630 samples
Epoch 1/10
2517/2517 [==============================] - 2s 984us/step - loss: 1.0422 - acc: 0.4474 - val_loss: 1.3279 - val_acc: 0.0000e+00
Epoch 2/10
2517/2517 [==============================] - 1s 514us/step - loss: 0.9494 - acc: 0.5491 - val_loss: 1.2867 - val_acc: 0.0000e+00
Epoch 3/10
2517/2517 [==============================] - 1s 510us/step - loss: 0.8762 - acc: 0.5952 - val_loss: 1.3321 - val_acc: 0.0095
Epoch 4/10
2517/2517 [==============================] - 1s 524us/step - loss: 0.8184 - acc: 0.6408 - val_loss: 1.2778 - val_acc: 0.0127
Epoch 5/10
2517/2517 [==============================] - 1s 500us/step - loss: 0.7642 - acc: 0.6611 - val_loss: 1.1915 - val_acc: 0.0381
Epoch 6/10
2517/2517 [==============================] - 1s 503us/step - loss: 0.7248 - acc: 0.6834 - val_loss: 1.1503 - val_acc: 0.0984
Epoch 7/10
2517/2517 [==============================] - 1s 508us/step - loss: 0.7040 - acc: 0.6921 - val_loss: 1.0607 - val_acc: 

In [31]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_count= tokenizer.texts_to_matrix(trainX,mode='count')
    X_test_matrix_count= tokenizer.texts_to_matrix(testX, mode='count')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_count= encoder.transform(trainy)
    Y_test_matrix_count= encoder.transform(testy)     
    
    smote = SMOTE('minority')
    X_sm_count, y_sm_count = smote.fit_sample(X_train_matrix_count, Y_train_matrix_count)
    
    model = Sequential()
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    model. summary()
    #print(model.metrics_names)
    from sklearn.utils import class_weight
    class_weight= class_weight.compute_class_weight('balanced', np.unique(rationale_type),rationale_type)
    #print(class_weight)
    model.fit(X_sm_count,y_sm_count, batch_size=batch_size, class_weight=class_weight, epochs=epochs, verbose=1, validation_split=0.3)
    score=model.evaluate(X_test_matrix_count, Y_test_matrix_count, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_count, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_count, axis=1), predicted)
    return model, test_accuracy, report_accuracy  

    

In [50]:
#Kfold ANN with mode=count

kf= KFold(n_splits=2)
#curr_fold= 0
alg_accurcy_ANN_count =[]
alg_testing_accurcy_ANN_count =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_count.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_count.append(report_accuracy)
    

average_accuracy_ANN_count= np.mean(alg_accurcy_ANN_count, axis=0)
print("the average Training Accuracy of ANN-Binary is",average_accuracy_ANN_count) 

average_accuracy_ANN_count= np.mean(alg_testing_accurcy_ANN_count, axis=0)
print("the averag Validation accuracy of ANN-Binary is",average_accuracy_ANN_count)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 16)                160016    
_________________________________________________________________
activation_63 (Activation)   (None, 16)                0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 3)                 51        
_________________________________________________________________
activation_64 (Activation)   (None, 3)                 0         
Total params: 160,067
Trainable params: 160,067
Non-trainable params: 0
_________________________________________________________________
Train on 1539 samples, validate on 660 samples
Epoch 1/10
1539/1539 [==============================] - 8s 5ms/step - loss: 1.0160 - acc: